In [ ]:
import os
file_dir = r"..\llm-20-questions"
origin_notebook_dir = file_dir + r"\kernels"
overview_dir = file_dir + "_jp"
overview_path = overview_dir + r"\overview.md"
output_dir = file_dir + "_jp"
output_discuss_dir = output_dir + r"\kernels"
if not os.path.exists(output_discuss_dir):
    os.makedirs(output_discuss_dir)

In [ ]:
import os

def list_directories(path):
    return [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
def list_extension_files(dirs,extension='.ipynb'):
    ipynb_files = []
    exit_indexs = []
    for i,dir in enumerate(dirs):
        with os.scandir(dir) as entries:
            for entry in entries:
                if entry.is_file() and entry.name.endswith(extension):
                    ipynb_files.append(entry.path)
                    exit_indexs.append(i)
                    break
    return ipynb_files,exit_indexs

notebook_dirs = list_directories(origin_notebook_dir)
output_dirs = [os.path.join(output_discuss_dir+"\\",dir) for dir in notebook_dirs]
for output_dir in output_dirs:
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
notebook_dirs = [os.path.join(origin_notebook_dir+"\\",dir) for dir in notebook_dirs]
notebook_files,exit_indexs = list_extension_files(notebook_dirs)
tmp,tmp_ = [],[]
for i in exit_indexs:
    tmp.append(notebook_dirs[i])
    tmp_.append(output_dirs[i])
notebook_dirs = tmp
output_dirs = tmp_
# notebook_filesのファイル名を取得
notebook_file_names = [os.path.basename(file) for file in notebook_files]
#output_dirsとnotebook_file_namesを結合
output_files = [os.path.join(output_dirs[i],notebook_file_names[i]) for i in range(len(notebook_files))]


In [ ]:
#output_dirsの中にファイルがないインデックス番号を取得
empty_dirs = [i for i in range(len(output_dirs)) if not os.listdir(output_dirs[i])]

#空の物のみを取得
output_files = [output_files[i] for i in empty_dirs]
notebook_files = [notebook_files[i] for i in empty_dirs]

In [ ]:
from calendar import c
from urllib import response
import nbformat
import os
import time
from tqdm import tqdm

def remove_python_markers(text):
    lines = text.split('\n')
    if lines[0] == '```python':
        lines = lines[1:]
    if lines[-1] == '```':
        lines = lines[:-1]
    return '\n'.join(lines)

class ResourceExhausted(Exception):
    pass
from prompt_toolkit import prompt
os.environ["GEMINI_API_KEY"] = "YOUR_API_KEY"
import google.generativeai as genai
genai.configure(api_key=os.environ["GEMINI_API_KEY"])
import re
from google.generativeai.types import HarmBlockThreshold, HarmCategory

generation_config = {
    "temperature": 0,
    "top_p": 0.95,
    "top_k": 64,
    "max_output_tokens": 8192,
    "response_mime_type": "text/plain",
}

with open(overview_path, 'r', encoding='utf-8') as file:
    content = file.read()
system_instruction = """あなたはプロの翻訳家兼プログラマーです。ユーザーから送られてくるkaggleの公開jupyter notebookの一部のセル群をそれぞれ日本語に翻訳するのがあなたの役割です。具体的にはマークダウンセル内の文章を日本語に翻訳したり、コードセル内のコードのコメントを日本語に翻訳してコメントが日本語になったコードを出力してください。ただし、コードセル内のコードそのものは絶対に変更や追記・削除やはしてはいけません。ただし追記の特例として、python初学者でもコード理解が出来るように、理解を促す簡潔なコメントをpythonのコメント形式#を用いて各コードの上に日本語で挿入し、理解を促すのは大丈夫です。むしろこれはあなたの役割なので、こちらに関しては積極的に行ってください。また、日本語に翻訳する際は単に直訳するのではなく、日本語として自然な文章になるよう心がけてください。これらの内容以外は出力せず、また出力の形式はユーザーの入力の形式と絶対に同じになるように、---Here, the cells separate. The following area is a {Markdown|Code} cell---\nで定義される特殊な文字列で区切ってからそれぞれ出力することを忘れないでください。---Here, the cells separate. The following area is a {Markdown|Code} cell---で囲まれた領域が一つのセルになるので、一つ抜けるだけで意味が大きく変わります。例えば、マークダウンセル、コードセル、マークダウンセル、コードセルの場合以下のような入力が与えられます。

ユーザーからの入力例(マークダウンセル、コードセル、マークダウンセル、コードセルの場合):
---Here, the cells separate. The following area is a Markdown cell---
{マークダウンセルの内容}
---Here, the cells separate. The following area is a Code cell---
{コードセルに書かれているコード}
---Here, the cells separate. The following area is a Markdown cell---
{マークダウンセルの内容}
---Here, the cells separate. The following area is a Code cell---
{コードセルに書かれているコード}

上記の入力に対して期待する出力例:
---Here, the cells separate. The following area is a Markdown cell---
{マークダウンセルの内容の日本語訳したもの}
---Here, the cells separate. The following area is a Code cell---
{コードセルに書かれているコメントを日本語訳し、さらに分かりやすくなるようにコメントを追加したもの}
---Here, the cells separate. The following area is a Markdown cell---
{マークダウンセルの内容の日本語訳したもの}
---Here, the cells separate. The following area is a Code cell---
{コードセルに書かれているコメントを日本語訳し、さらに分かりやすくなるようにコメントを追加したもの}

また、チャットの履歴は、送った内容以前のセルです。つまりユーザーから入力されたセル以前より前の内容なので内容としてはつながっているので適宜参考にしてください。また、コンペティションの概要は以下の通りです。文脈の理解に必要であれば用いて構いません。

コンペティションの概要:
```markdown
""" + content + "\n```"

model = genai.GenerativeModel(
    model_name="gemini-1.5-pro",
    generation_config=generation_config,
    system_instruction=system_instruction,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    }

)

chat_session = model.start_chat(history=[])

#リクエストの処理
def send_request(chat_session,prompt):
    try:
        response = chat_session.send_message(prompt)
    except ResourceExhausted as e:
        print("Resource exhausted. Waiting for 1 minute before retrying...")
        for i in tqdm(range(60), desc="Waiting", unit="second"):
            time.sleep(1)
        response = chat_session.send_message(prompt)
    except Exception as e:
        print(f"An error occurred: {e}")
        raise
    return response

def merge_markdown_cells(input_path, output_path=None):
    with open(input_path, 'r', encoding='utf-8') as f:
        notebook = nbformat.read(f, as_version=4)

    new_cells = []
    current_markdown = ""

    for cell in notebook.cells:
        if cell.cell_type == 'markdown':
            if current_markdown:
                current_markdown += "\n\n"  # 各マークダウンセル間に空行を追加
            current_markdown += cell.source
        else:
            if current_markdown:
                new_cells.append(nbformat.v4.new_markdown_cell(current_markdown))
                current_markdown = ""
            new_cells.append(cell)

    # 最後のマークダウンセルがある場合、それを追加
    if current_markdown:
        new_cells.append(nbformat.v4.new_markdown_cell(current_markdown))

    notebook.cells = new_cells
    if output_path:
        with open(output_path, 'w', encoding='utf-8') as f:
            nbformat.write(notebook, f)
    return notebook

def translate_batch(batch,historys=[]):
    historys_ = []
    for history in historys:
        user_history = {"role": "user", "parts": history[0]}
        model_history = {"role": "model", "parts": history[1]}
        historys_.append(user_history)
        historys_.append(model_history)
    chat_session = model.start_chat(history=historys_)
    #
    input_text = "\n".join([f"---Here, the cells separate. The following area is a {cell_type} cell---\n{content}" for cell_type, content in batch])
    print("モデルに送るテキスト")
    print(input_text)
    response = send_request(chat_session,input_text)
    print("モデルからの返答")
    print(response.text)
    historys.append([input_text,response.text])
    # ---Markdown---\nまたは---Code---\nで区切る。正規表現で分割する
    response_parts = re.split(r'---Here, the cells separate. The following area is a (Markdown|Code) cell---\n', response.text)
    response_parts = response_parts[1:]  # 最初の要素は空文字列なので除外
    # 奇数番目だけ取り出す
    response_parts = response_parts[1::2]
    return response_parts,historys

def get_md_files_in_same_directory(path):
    # 指定されたパスのディレクトリを取得
    directory = os.path.dirname(path)
    
    # 指定されたディレクトリ内のファイルとディレクトリのリストを取得
    files_and_dirs = os.listdir(directory)
    
    # .mdファイルのリストを作成
    md_files = [os.path.join(directory, f) for f in files_and_dirs if f.endswith('.md')]
    
    return md_files
def translate_notebook(notebook_path, output_path, max_tokens=3000):
    with open(notebook_path, 'r', encoding='utf-8') as f:
        notebook = nbformat.read(f, as_version=4)
    md_paths = get_md_files_in_same_directory(notebook_path)
    if len(md_paths) > 0:
        md_path = md_paths[0]
        with open(md_path, 'r', encoding='utf-8') as f:
            content = f.read()
            content = "---\n\n" + content
            #ノートブックの末尾に追加
            content_cell = nbformat.v4.new_markdown_cell(content)
            notebook.cells.append(content_cell)

    cells_to_translate = []
    for cell in notebook.cells:
        if cell.cell_type in ['markdown', 'code']:
            cells_to_translate.append((cell.cell_type.capitalize(), cell.source))

    translated_cells = []
    historys = []
    current_batch = []
    current_tokens = 0
    # 累積トークン数(最後に表示するため)
    total_tokens = 0
    model = genai.GenerativeModel( # 文字数カウント用
        model_name="gemini-1.5-pro",
        generation_config=generation_config,
        safety_settings={
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        }
    )
    for cell_type, content in cells_to_translate:
        cell_text = f"---Here, the cells separate. The following area is a {cell_type} cell---\n{content}"
        cell_tokens = model.count_tokens(cell_text).total_tokens

        if current_tokens + cell_tokens > max_tokens and current_batch:
            # 現在のバッチを翻訳
            total_tokens += current_tokens
            translated_batch, historys = translate_batch(current_batch, historys)
            translated_cells.extend(translated_batch)
            current_batch = []
            current_tokens = 0

        current_batch.append((cell_type, content))
        current_tokens += cell_tokens

    # 残りのセルを処理
    if current_batch:
        total_tokens += current_tokens
        translated_batch, historys = translate_batch(current_batch, historys)
        translated_cells.extend(translated_batch)
    translated_index = 0
    print(len(translated_cells))
    print(len([cell for cell in notebook.cells if cell.cell_type in ['markdown', 'code']]))
    for cell in notebook.cells:
        if cell.cell_type == 'markdown' or cell.cell_type == 'code':
            print(translated_index)
            if cell.cell_type == "code":
                cell.source = remove_python_markers(translated_cells[translated_index].strip())
            else:
                cell.source = translated_cells[translated_index]
            translated_index += 1
        # その他のセルタイプ（出力、図など）はそのまま保持
    print(f"Total tokens: {total_tokens}")
    with open(output_path, 'w', encoding='utf-8') as f:
        nbformat.write(notebook, f)



# ノートブックの要約を生成
def summarize_notebook(notebook_path):
    with open(notebook_path, 'r', encoding='utf-8') as f:
        notebook = nbformat.read(f, as_version=4)

    text = ""
    for cell in notebook.cells:
        if cell.cell_type == 'markdown':
            text += "マークダウンセル :\n"+cell.source+"\n\n"
        elif cell.cell_type == 'code':
            text += "コードセル :\n"+cell.source+"\n\n"
    system_instruction = """あなたはプロのプログラマーです。ユーザーから送られてくるkaggleの公開jupyter notebookに関するタスクを遂行してください。また、コンペティションの概要は以下の通りです。文脈の理解に必要であれば用いて構いません。

    コンペティションの概要:
    ```markdown
    """ + content + "\n```"
    model = genai.GenerativeModel(
        model_name="gemini-1.5-pro",
        generation_config=generation_config,
        system_instruction=system_instruction,
        safety_settings={
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        }
    )
    chat_session = model.start_chat(history=[])
    total_tokens = 0
    prompt_summary = "以下に示すjupyter notebookの要約を生成してください。特にそのnotebookはどのような問題に取り組み、どのような手法やライブラリで解決しているのかは必ず要約に含めてください。\n\n ノートブックの内容 :\n"+text
    prompt_know = "以下に示すjupyter notebookのを読んで、機械学習・深層学習の初心者がつまずきそうな専門用語の簡単な解説を列挙してください。ただし、初心者なだけで大学の学部でやるようなレベルの機械学習・深層学習の知識はあるので簡単なものや有名なものは説明不要です。ある程度マイナーなものや、実務を経験していないと馴染みのないもの、このノートブック特有のドメイン知識等に焦点を当ててください。\n\n ノートブックの内容 :\n"+text
    total_tokens += model.count_tokens(prompt_summary).total_tokens
    response = send_request(chat_session,prompt_summary)
    summary = response.text
    summary = "# 要約 \n"+summary+"\n\n---\n"
    # 要約を既存のnotebookの先頭にマークダウンセルとして追加
    summary_cell = nbformat.v4.new_markdown_cell(summary)
    notebook.cells.insert(0, summary_cell)
    total_tokens += model.count_tokens(prompt_summary).total_tokens
    response = send_request(chat_session,prompt_know)
    known = response.text
    known = "# 用語概説 \n"+known+"\n\n---\n"
    # 要約を既存のnotebookの先頭にマークダウンセルとして追加
    known_cell = nbformat.v4.new_markdown_cell(known)
    notebook.cells.insert(1, known_cell)
    print(f"Total tokens: {total_tokens}")
    with open(notebook_path, 'w', encoding='utf-8') as f:
        nbformat.write(notebook, f)
    return summary


In [ ]:
for i in range(len(notebook_files)):
    _ = merge_markdown_cells(notebook_files[i], notebook_files[i])
    _ = translate_notebook(notebook_files[i], output_files[i])
    _ = summarize_notebook(output_files[i])
